In [1]:
TITLES_MANAGEMENT_MODIFICATION_USER_PROMPT = """
<Information>
    - We are querying our database based on job titles, management levels extracted from a user's input. Your focus would only be on job titles and management levels mentioned in the prompt. 
    - Management levels can only be one of the following predefined options:
        ["Partners,"," "Founder or Co-founder," "Board of Directors," "CSuite/Chiefs," "Executive VP or Sr. VP," "General Manager," "Manager," "Senior Partner," "Junior Partner," "VP," "Director," "Senior (All Senior-Level Individual Contributors)," "Mid (All Mid-Level Individual Contributors)," "Junior (All Junior-Level Individual Contributors)"].
    - Job titles can be any logical job title that a person can have.

    <Instruction>    
        A management level is chosen only if it encompasses the entire domain. 
            - Example: "Board Chair" is a subset of "Board of Directors," so it will be treated as a job title. VP of Engineering" is a subset of "VP," so it will also be treated as a job title.
            - Example: "Director of Microsoft" or "Directors in the automotive industry" lacks business function specificity and will default to 'Director' as a management level.

        Key phrases in a prompt refer to titles and functions (if mentioned). If a function is specified, it is classified as a "Job Title." For example:
            - Query: "CEOs working in the automotive industry and VP of Engineering at Microsoft."
                Here, "CEOs" and "VP of Engineering" are key phrases. Only "VP of Engineering" is treated as a job title due to its specificity, while "VP" cannot stand alone as a key phrase.
            - Exclude company names, industries, or non-logical phrases from key phrases. (Industries, companies, skills, etc. will not be included in Key phrases that are to be judged)
            - A key phrase should be categorized as a Job Title or Management Level. IT MUST NEVER BE CLASSIFIED INTO BOTH.
            - The 'Manager' level will not be selected when a specific type of manager (e.g., Senior Manager, Project Manager, etc.) is requested. Similarly, Junior, Senior, and Mid-level individual contributors will ONLY be chosen when ALL contributors at that level (e.g., all Junior, all Senior, or all Mid-level individual contributors) are requested, not for specific roles like Senior Software Engineer or Mid-level Management.

        Rules for classification:
            - Normalize the job titles and give all known variations of a title (dont change the title). For example,  and always include both the full title and its variations (confirmed ones), e.g., "VP of Engineering", "Vice President of Engineering.", "VP Engineering", etc. or "Chief Innovation Officer" and "CINO" as separate elements. ENSURE LOGICAL and EXACT job titles such as 'Architect' NOT 'Architect who is skilled in VR' and NOT 'Architect in automotive' (industries, companies, etc will not come in job titles). Job titles MUST BE CONCISE AND TO THE POINT and shouldn't include company names or region names.
            - If a user wants all 'executives', without any business function specified then three management levels will be covered, namely "CSuite," "Executive VP or Sr. VP" and "VP" so all MUST come. However, the word 'executive' is mentioned in relation to a business function, only titles specific to that function should be included. For example, if 'Marketing Executives' is mentioned, titles such as 'CMO', 'Chief Marketing Officer', 'Senior VP of Marketing', 'Senior Vice President of Marketing', 'VP of Marketing', and 'Vice President of Marketing' should be included. The word 'executive' or 'executives' would, thus, NEVER be included neither as job title nor management level.

        Claude has to perform the following four tasks:
            1. Claude has to find keyphrases in the prompt and classify them into job titles and management levels.  For the query "Find VPs and CEOs", CEO would be a job title while VP would be the management level. This step is like an NER step.
            2. If neither job titles nor management levels have been mentioned explicitly, Claude has to analyze which job titles can be inferred from the user's prompt, if any (if job titles are inferred and not explicitly mentioned, then management levels should not be selected)
            3. Claude has to classify each management level and job title in the correct timeline.
            4. Claude has to generate similar titles for every non-executive title (below director-level) in all cases, but for executive roles, similar titles will only be generated if the query looks like its for a hiring purpose or asks for similar titles of executives. Similar titles would be of similar ranks and and similar business functions as required by the query. (If added, similar titles will be added in the same format as the original titles)
            5. If something is being reiterated or reinforced by the user, make changes accordingly.
    </Instruction>
    <Format>
        {
            "job_titles" : {
                "Current" : [{"name" : "..", "min" : "..", "max" : ".."}..],
                "Past" : [{"name" : "..", "min" : "..", "max" : ".."}..],
                "Both" : [{"name" : "..", "min" : "..", "max" : ".."}..],
                "Event" : ".."
            },
            "management_levels" : {
                "Current" : [],
                "Past" : [],
                "Both" : [],
                "Event" : ".."
            },
            "reasoning" : '''...''' # triple-quoted string
        }
        Key: job_titles:
            Contains four keys: Current, Past, Both, and Event.

            Each key represents a temporal context for job titles:

            - Current: Refers to people who are currently holding the specified title.
            - Past: Refers to people who have held the specified title in the past but no longer hold it.
            - Both: Refers to people who currently hold the title and also those who have held it in the past.
              Example: For a query like "Software engineers who have ever been in the position of CFO," 'CFO' should be classified as 'Both' because it implies interest in both current and past CFOs. Conversely, 'Software Engineer' should be classified as 'Current,' as it specifies the current role. 
            - Event: Can only be one of ["Current", "Past", "AND", "OR"]. If all titles fall exclusively under 'Current,' select 'Current.' If all titles fall exclusively under 'Past,' select 'Past.' If any title falls under 'Both' or there's a mix of 'Current,' 'Past,' and 'Both,' then choose 'AND' (sequential roles required) or 'OR' (flexible roles required). This applies strictly to job titles, not management levels.

            For Current, Past, and Both, the data structure is a list of JSON objects, where each object contains:
                - min: The minimum company staff size (default is 1 if unspecified).
                - max: The maximum company staff size (default is 50,000,000 if unspecified).
                - name: The job title.

        Key: management_levels:
            Contains four keys: Current, Past, Both, and Event.

            - Current: Refers to people currently at the specified management level.
            - Past: Refers to people who were at the specified management level in the past but no longer hold it.
            - Both: Refers to people who are currently at the level and also those who have been at the level in the past.
              Example: For a query like "Find Managers who have ever been chiefs of a company," classify 'chiefs' as 'Both' because it implies interest in both current and past chiefs. Conversely, 'Manager' would be 'Current,' as it specifies the current level.
            - Event: Can only be one of ["Current", "Past", "AND", "OR"]. Follow the same logic as the Event key under job_titles, but this applies strictly to management levels.

            Data structure:
            These keys hold a list of strings specifying the management levels.
        
        Key: reasoning:
            All your reasoning should be explained in the reasoning key, including why similar titles were/weren't added.
    </Format>

    Return the payload (JSON object enclosed inside <Output></Output> tags is a essential in output) with any required changes in the exact same JSON format as explained above.

"""

In [2]:
NER_MANAGEMENT_LEVEL_TITLE_SYSTEM_PROMPT = """
You are an intelligent assistant dedicated to extracting management levels and job titles from user queries. Before doing so, you must understand what a functional area is.
"""

NER_MANAGEMENT_LEVEL_TITLE_USER_PROMPT = """

Instructions:
1. Management Levels: Only return management levels that match the predefined set: ["Partners,"," "Founder or Co-founder," "Board of Directors," "CSuite/Chiefs," "Executive VP or Sr. VP," "General Manager," "Manager," "Head," "Senior Partner," "Junior Partner," "VP," "Director," "Senior (All Senior-Level Individual Contributors)," "Mid (All Mid-Level Individual Contributors)," "Junior (All Junior-Level Individual Contributors)"]. MANAGEMENT CAN ONLY BE FROM THIS PREDEFINED SET, Nothing ELSE.
2. Job Titles: Normalize the job title after extracting it from the text. For example, convert "ceo" to "Chief Executive Officer" and always include both the full title and its abbreviation (confirmed ones), e.g., "VP of Engineering" and "Vice President of Engineering." or "Chief Innovation Officer" and "CINO". ENSURE LOGICAL and EXACT job titles such as 'Architect' NOT 'Architect who is skilled in VR'. Job titles MUST BE CONCISE AND TO THE POINT and shouldn't include company names or region names. Do not change the title for normalization.
3. Response Format: Your response must be a dictionary with two keys: "management_levels" and "titles". Each key should have a list of management levels and titles respectively.
4. Identify the key phrases in a prompt. Key phrase is a title and its function, IF THE function is mentioned. If the function would be mentioned, it will be classified as a "Job Title". For example, "CEOs working in Automotive Industry and VP of Engineering of Microsoft" has the "CEOs" and "VP of Engineering" as Key Phrases. In this case ONLY VP CANNOT be considered a key phrase. ONLY CONSIDER THE KEY PHRASES MENTIONED, DO NOT ASSUME. Past and current designations dont matter.
5. Check whether the KEY PHRASE should be classified as a Job Title or a Management Level. IT SHOULD NEVER BE CLASSIFIED INTO BOTH. This is ESSENTIAL.
5. If a key phrase is classified as title, don't include it in the management levels. For example, if "VP of Engineering" is classified as title then don't include "VP" in management levels. Industry or company names will not be included in job titles.
6. If a key phrase is classified as a management level, don't include it in the title. For example, if "Vice Presidents" is classified as management level then don't include "Vice Presidents" in titles. A job title will ONLY be a title and its business function. No other DETAIL should be added. A Management Level cannot COME FROM WITHIN A JOB TITLE.
7. Remember: One instance of a key phrase should be considered for either management level or job title, not both. Each will fall either into management levels or job title but WILL NEVER FALL INTO BOTH. A KEY PHRASE CANNOT BE IN MANAGEMENT LEVELS AND TITLE, BOTH. A Management Level cannot COME FROM WITHIN A JOB TITLE. 
8. If the word 'executive' is mentioned, specific considerations should be taken into account.

Take a deep breath and understand.
Query: "Give me VPs working in Microsoft": # VPs is the KEY PHRASE. It will be evaluated as a whole key phrase ONLY. It cannot be classified into a management level and a title both.
Management Level Focus: In this query, "VPs" should refer to individuals at the management level of Vice President within Microsoft. This means you are asking for a list of people who occupy the VP rank across various departments or divisions within the company. VPs can cover the complete domain of of 'VP' in management level. The emphasis is on their standing in the organizational hierarchy, regardless of their specific job titles. A management title will only be selected if it covers the complete domain in the predefined set.
Job Title Focus: If you were asking about "VPs" in terms of job titles, you'd be interested in individuals whose specific title is "VP" of a certain business function, such as "VP of Marketing" (Marketing is a business function) or "VP of Engineering" (Engineering is a business function). If a function is clearly mentioned then it would be JOB TITLE. "VP of Microsoft" does NOT have a function (Microsoft is an organization) neither does "Automotive VPs" (Automotive can ONLY be an industry). Identify the BUSINESS functions accurately. Then they CANNOT come under management levels.
Output: {"management_levels": ["VP"], "titles": []}

Query: "The CFOs working in google or facebook": #CFOs is a KEY PHRASE. It will be evaluated as a whole key phrase ONLY. It cannot be classified into a management level and a title both.
Management Level Focus: In this query, "CFOs" does not cover the complete 'C-Suite' domain. ONLY IF COMPLETE DOMAIN IS COVERED then the key phrase will be in management level. One job title, even if it is on the top or head of the heirarchy, does not cover the complete domain. If a user wants all 'executives', without any business function specified then three management levels will be covered, namely "CSuite," "Executive VP or Sr. VP" and "VP" so all MUST come. However, the word 'executive' is mentioned in relation to a business function, only titles specific to that function should be included. For example, if 'Marketing Executives' is mentioned, titles such as 'CMO', 'Chief Marketing Officer', 'Senior VP of Marketing', 'Senior Vice President of Marketing', 'VP of Marketing', and 'Vice President of Marketing' should be included. The word 'executive' or 'executives' would, thus, NEVER be included neither as job title nor management level.
Job Title Focus: As a CFO would only be a chief in finance (not covering the CSuite domain), the CFO being discussed here comes under job title, not management level, same if the title was "heads of finance" (wouldn't cover Head domain), etc.
Output: {"management_levels": [], "titles": ["CFO", "Chief Finance Officer"]} # Job titles MUST BE CONCISE and TO THE POINT, mentioning ONLY the TITLE and the BUSINESS FUNCTION if the business function is given. No added details, such as company name or group.

If terms like 'leader', 'expert', 'specialist', or similar are mentioned, extract a maximum of 2-3 relevant job titles associated with those terms based on the context, focusing on the most appropriate leadership or expertise roles. However, do not extract titles based on names, companies, etc.

For each management level and title, also tell why you put it there. If a business function can be clearly identified, the key phrase will be a JOB title. ALWAYS make the necessary changes when the word 'executive' or 'leader' or 'expert', etc., is mentioned in the user query and get LOGICAL titles. Management level of 'Manager' will not be chosen when a specific type of 'manager' (senior managers, project manager, etc.) are asked for. ONLY identify and consider complete key phrases EXPLICITLY MENTIONED IN THE PROMPT, and each key phrase will either be in management level or title, NEVER consider THE SAME KEY PHRASE for BOTH. Evaluate key phrases separately.
Always return a JSON object in your output.

"""